In [ ]:
# default_exp display
# hide
_FNAME='display'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os
TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')
notebook2script(_nbpath)

Converted display.ipynb.


In [ ]:
#export
import discord


[discord.Message.created_at returns a datetime.datetime](https://discordpy.readthedocs.io/en/stable/api.html#discord.Message.created_at)


In [ ]:
from datetime import datetime
testts = datetime(year=2021, month=12, day=7, hour=12, minute=31, second=15, microsecond=12345 )
testts.strftime("%b %d %Y %H:%M:%S")

'Dec 07 2021 12:31:15'

In [ ]:
#export

import json


def encode(u):
    if isinstance(u, discord.Message):
        ts = u.created_at
        serialized = "({ts}){author}: {content}".format(ts=ts.strftime("%b %d %Y %H:%M:%S"), 
                                                        author=u.author.name, 
                                                        content=u.content)
        return serialized
    elif isinstance(u, discord.Thread):
        return 'Thread: {}'.format(u.name)
    elif isinstance(u, discord.TextChannel):
        return 'Channel: {}'.format(u.name)
    elif isinstance(u, discord.Guild):
        return 'Guild: {}'.format(u.name)
    else:
        type_name = u.__class__.__name__
        raise TypeError("Unexpected type {0}".format(type_name))

class DiscordEncoder(json.JSONEncoder):
    def default(self, u):
        if isinstance(u, discord.Message):
            """
            serialized = {
                "id": u.id,
                "content": u.content,
                "author": u.author.name,
                "created_at": u.created_at.isoformat()
            }
            """
            serialized = "({ts}){author}: {content}".format(ts=u.created_at.isoformat(), 
                                                            author=u.author.name, 
                                                            content=u.content)
            return serialized
        elif isinstance(u, discord.Thread):
            return 'Thread: {}'.format(u.name)
        elif isinstance(u, discord.TextChannel):
            return 'Channel: {}'.format(u.name)
        elif isinstance(u, discord.Guild):
            return 'Guild: {}'.format(u.name)
        else:
            #type_name = u.__class__.__name__
            #raise TypeError("Unexpected type {0}".format(type_name))
            return json.JSONEncoder.default(self, obj)
            
            
class Formatter:
    def __init__(self):
        self.lines = []
        
    def add(self, thing):
        #entry = json.dumps(thing, cls=DiscordEncoder)
        entry = encode(thing)
        self.lines.append(entry)


In [ ]:
#export

#TODO change the data model for this to something more standard.
# use only strings for the keywords rather than discord objects
def serialize_content(guild_content):
    fmt = Formatter()
    print('--------- content summary -------------')
    for guild, channels_d in guild_content.items():
        fmt.add(guild)
        for channel_obj, thread_d in channels_d.items():
            fmt.add(channel_obj)
            for thread, msg_list in thread_d.items():
                if msg_list:
                    fmt.add(thread)
                for msg in msg_list:
                    fmt.add(msg)
    return fmt.lines

def html_content(guild_content):
    lines = serialize_content(guild_content)
    print(lines)
    return '\n<br>'.join(lines)